# Context extraction

For usage in agent (see `07_agent.ipynb` notebook) we should provide:
- information about user and his goals
- thread representation / message history
- information about context

One of typical kind of contexts may be social networks of all kind. For them post can be a context and comments can be threads.

So we can make a goal-oriented summary of this post.

## Implementation

### Imports

In [ ]:
#| default_exp context_extractor

In [ ]:
#| export
import os
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chains import TransformChain
from langchain.chat_models.base import BaseChatModel
from langchain.chat_models.openai import ChatOpenAI
from pino_inferior.core import PROMPTS_DIR, OPENAI_API_KEY
from datetime import datetime
from typing import Callable
from langchain.schema import AIMessage
from langchain.schema.runnable import RunnableSequence
import tiktoken
from dataclasses import dataclass

In [ ]:
#| export
CONTEXT_PROMPT_DIR = os.path.join(PROMPTS_DIR, "context")

In [ ]:
#| export
CONTEXT_INPUT_TEXT = "text"
CONTEXT_INPUT_POST_TIME = "post_datetime"
CONTEXT_INPUT_GOALS = "goals"
CONTEXT_INPUT_CURRENT_TIME = "current_datetime"
CONTEXT_INPUT_USERNAME = "name"
CONTEXT_INPUT_CHARACTER = "character"

CONTEXT_INTERMEDIATE_POST_CUTTEN = "post_cutten"
CONTEXT_INTERMEDIATE_POST_TIME_STR = "post_datetime_str"
CONTEXT_INTERMEDIATE_CURRENT_TIME_STR = "current_datetime_str"
CONTEXT_INTERMEDIATE_LLM_OUTPUT = "llm_output"

CONTEXT_OUTPUT_CONTEXT = "context"

### Conversions

In [ ]:
#| export
def _read_file(fname: str) -> str:
    with open(fname, "r", encoding="utf-8") as src:
        return src.read()

In [ ]:
#| export
context_system_prompt = SystemMessagePromptTemplate.from_template(_read_file(
    os.path.join(CONTEXT_PROMPT_DIR, "system.txt")
))
context_instruction_prompt = HumanMessagePromptTemplate.from_template(_read_file(
    os.path.join(CONTEXT_PROMPT_DIR, "instruction.txt")
))
context_llm_prompt = ChatPromptTemplate.from_messages([context_system_prompt, context_instruction_prompt])

In [ ]:
#| export
def _build_preprocess_conversion(length_function: Callable[[str], int],
                                 cut_function: Callable[[str, int], int],
                                 max_goals_length: int,
                                 max_name_length: int,
                                 max_character_length: int,
                                 max_post_length: int,) -> Callable[[dict], dict]:
    def _func(row: dict) -> dict:
        post_time: datetime = row[CONTEXT_INPUT_POST_TIME]
        current_time: datetime = row[CONTEXT_INPUT_CURRENT_TIME]
        goals: str = row[CONTEXT_INPUT_GOALS]
        name: str = row[CONTEXT_INPUT_USERNAME]
        character: str = row[CONTEXT_INPUT_CHARACTER]
        assert length_function(goals) <= max_goals_length
        assert length_function(name) <= max_name_length
        assert length_function(character) <= max_character_length
        post: str = row[CONTEXT_INPUT_TEXT]
        if length_function(post) > max_post_length:
            post = cut_function(post, max_post_length)
        return {
            CONTEXT_INTERMEDIATE_CURRENT_TIME_STR: current_time.strftime("%d %b %Y %H:%M"),
            CONTEXT_INTERMEDIATE_POST_TIME_STR: post_time.strftime("%d %b %Y %H:%M"),
            CONTEXT_INTERMEDIATE_POST_CUTTEN: post,
        }
    
    async def _afunc(row: dict) -> dict:
        return _func(row)
    
    return TransformChain(transform=_func,
                          atransform=_afunc,
                          input_variables=[CONTEXT_INPUT_CURRENT_TIME,
                                           CONTEXT_INPUT_POST_TIME,
                                           CONTEXT_INPUT_GOALS,
                                           CONTEXT_INPUT_TEXT],
                          output_variables=[CONTEXT_INTERMEDIATE_POST_TIME_STR,
                                            CONTEXT_INTERMEDIATE_CURRENT_TIME_STR,
                                            CONTEXT_INTERMEDIATE_POST_CUTTEN])

In [ ]:
#| export
def _build_llm_output_parser(
    length_function: Callable[[str], int],
    cut_function: Callable[[str, int], str],
    max_context_length: int,
    tags_open_marker: str,
    tags_close_marker: str,
    summary_open_marker: str,
    summary_close_marker: str
) -> TransformChain:
    def _func(response: dict) -> dict:
        ai_message: AIMessage = response[CONTEXT_INTERMEDIATE_LLM_OUTPUT]
        tags = None
        if tags_open_marker in ai_message.content and tags_close_marker in ai_message.content:
            tags = ai_message.content.split(tags_open_marker)[-1].split(tags_close_marker)[0]
        summary = ai_message.content.split(summary_open_marker)[-1].split(summary_close_marker)[0]
        response = f"[tags]{tags}[/tags]\n{summary}"
        if length_function(response) > max_context_length:
            response = cut_function(response, max_context_length)
        return {CONTEXT_OUTPUT_CONTEXT: response}
    
    async def _afunc(response: AIMessage) -> dict:
        return _func(response)
    
    return TransformChain(transform=_func, atransform=_afunc,
                          input_variables=[CONTEXT_INTERMEDIATE_LLM_OUTPUT],
                          output_variables=[CONTEXT_OUTPUT_CONTEXT])

In [ ]:
#| export
def _build_context_extractor(llm: BaseChatModel,
                             length_function: Callable[[str], int],
                             cut_function: Callable[[str, int], int],
                             max_goals_length: int,
                             max_name_length: int,
                             max_character_length: int,
                             max_post_length: int,
                             max_context_length: int,
                             tags_open_marker: str,
                             tags_close_marker: str, 
                             summary_open_marker: str,
                             summary_close_marker: str) -> RunnableSequence:
    stringify = _build_preprocess_conversion(
        length_function,
        cut_function,
        max_goals_length,
        max_name_length,
        max_character_length,
        max_post_length,
    )
    output_parser = _build_llm_output_parser(
        length_function,
        cut_function,
        max_context_length,
        tags_open_marker,
        tags_close_marker,
        summary_open_marker,
        summary_close_marker,
    )
    return stringify | context_llm_prompt | llm | output_parser

### Context extractor wrapper

In [ ]:
#| export
@dataclass
class LengthConfig:
    """
    Context extractor text length config
    """
    cut_function: Callable[[str, int], str]
    length_function: Callable[[str], int]
    max_goals_length: int = 256
    max_name_length: int = 10
    max_character_length: int = 256
    max_post_length: int = 2048
    max_response_length: int = 512


@dataclass
class PromptMarkupConfig:
    """
    Context extractor parsing sequences (described in prompt)
    """
    tags_open_sequence: str = "[tags]"
    tags_close_sequence: str = "[/tags]"
    summary_open_sequence: str = "[summary]"
    summary_close_sequence: str = "[/summary]"


def build_context_extractor_chain(llm: BaseChatModel, lengths: LengthConfig, prompts: PromptMarkupConfig) -> RunnableSequence:
    """
    Build context extraction LLM chain
    """
    return _build_context_extractor(
        llm,
        length_function=lengths.length_function,
        cut_function=lengths.cut_function,
        max_goals_length=lengths.max_goals_length,
        max_name_length=lengths.max_name_length,
        max_character_length=lengths.max_character_length,
        max_post_length=lengths.max_post_length,
        max_context_length=lengths.max_response_length,
        tags_open_marker=prompts.tags_open_sequence,
        tags_close_marker=prompts.tags_close_sequence,
        summary_open_marker=prompts.summary_open_sequence,
        summary_close_marker=prompts.summary_close_sequence,
    )

In [ ]:
llm = ChatOpenAI(
    model="gpt-4-0613",
    openai_api_key=OPENAI_API_KEY,
)
llm.__class__

langchain.chat_models.openai.ChatOpenAI

In [ ]:
encoding = tiktoken.encoding_for_model(llm.model_name)
encoding

<Encoding 'cl100k_base'>

In [ ]:
ce = build_context_extractor_chain(llm,
                                   lengths=LengthConfig(
                                       cut_function=lambda text, length: encoding.decode(encoding.encode(text)[:length]),
                                       length_function=lambda text: len(encoding.encode(text)),
                                       max_goals_length=512,
                                       max_name_length=10,
                                       max_character_length=512,
                                       max_post_length=2048,
                                   ),
                                   prompts=PromptMarkupConfig(
                                       tags_open_sequence="[tags]",
                                       tags_close_sequence="[/tags]",
                                       summary_open_sequence="[summary]",
                                       summary_close_sequence="[/summary]",
                                   ))
ce.__class__

langchain.schema.runnable.base.RunnableSequence

In [ ]:
TEXT = """Будет ли мобилизация осенью?⁠⁠
Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.

Мой канал в ТГ: https://t.me/artjockey

Напомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.

Будет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост
Первая волна мобилизации
Она началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять, а просто, опираясь на действующий указ, начать новый призыв.

Для чего нужна была мобилизация? Численность группировки вторжения ВС РФ была около 200 тысяч человек (с учетом войск еще независимых ДЛНР). Украинская армия мирного времени – примерно 260 тысяч человек. То есть, силы были примерно равны, даже в пользу ВС РФ, потому что у ВСУ это не вся сухопутная армия, а с учетом всех тыловых служб, штаба, пограничников и т.д.

Но сразу после начала войны, украинское правительство объявляет всеобщую мобилизацию, вполне логичный и правильный шаг. Численность украинской армии быстро увеличивается и к октябрю достигает 700 тысяч человек, по словам украинского министра обороны. Конечно, далеко не все эти люди находились на передовой, то есть, не являлись «штыками». Но перекос в численности воюющих армий уже очевиден и было понятно, что продолжать завоевательную войну имея в 2-3 раза меньше людей невозможно.

Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил. Даже если перекос в чью-то сторону сохранялся, он был уже не в разы, а в десятки процентов, не более. Это позволило и равномерно укрепить фронт, чтобы не допустить повторения сценария наступления ВСУ на Изюм.

Но всеобщая мобилизация на территории Украины не прекратилась, украинская армия продолжила пополняться, зимой стали известны и планы Киева провести наступление в 2023-м году. В таких условиях уже было понятно, что вторая волна – неизбежна. ВС РФ необходимо, хотя бы, поддерживать паритет по численности. Многие источники писали, что мобилизация будет проведена в начале года, чтобы иметь обученные резервы к концу весны-лету. Я тоже думал, что мобилизацию объявят.

Стратегия Кремля
Как мы знаем, вторую волну мобилизации не объявили и связано это с курсом руководства России на изоляцию войны от основной массы российского общества. Я не сомневаюсь, что подготовка действительно велась, но в какой-то момент планы поменялись.

Для среднего россиянина война идет и, по этой стратегии, должна идти где-то в телевизоре, потому и не война, а СВО. Да, есть некий процент людей, кого затронула мобилизация в первую волну, но их, относительно общего населения страны, немного.

Также временно вопрос пополнения закрыл собой Вагнер, через который, как мы теперь знаем, прошло 78 тысяч человек, что довольно много. Это где-то 25% от частично мобилизованных.

Но общая проблема с поддержкой паритета по численности личного состава никуда не делась и ее нужно было как-то решать. Решение было озвучено весной, вместо второй волны мобилизации предполагается до конца года набрать 400 тысяч людей на контрактную службу. Вот эти новые контрактники и должны заменить собой «вторую волну».

Скрытая мобилизация
Есть такое понятие, как скрытая мобилизация. То есть, набор в войска не обязательно ведется с официальными объявлениями по телевизору, фанфарами и всесторонним освещением в СМИ.

При этом, не стоит понимать термин настолько буквально, что это некие тайные мероприятия, о которых никто не должен узнать. Совсем не обязательно. Это просто некий общий комплекс мер, призванный замаскировать или скрыть отдельные детали процесса или же прикрыть его юридически. Так, например, официальная мобилизация государства может являться поводом для объявления войны, а, скажем, увеличение численности пограничных войск и парамилитарных образований – таким поводом уже не будет.

Также, скрытая мобилизация может скрывать не сам процесс, а его детали. При частичной мобилизации все знали и сколько людей будет призвано, и куда они будут направлены. При скрытой мобилизации такие подробности могут не озвучиваться, хотя сам проходящий процесс и является достоянием общественности.

Вот нынешняя стратегия российского руководства и является проведением скрытой мобилизацией. Официально, контрактники набираются не для участия в СВО, а согласно стратегическому плану развития армии, которое не так давно было подписано Путиным и предусматривает увеличение численности ВС РФ и вне контекста войны.

Методы скрытой мобилизации
Сейчас в РФ задействовано три основных источника пополнения личного состава.

Во-первых, это прямой поток добровольцев, заключающих контракты на службу в рядах армии. Жители России, думаю, видели и агрессивную рекламу службы, кто-то сталкивался с агитаторами на работе, а кто-то даже получал повестку для уточнения данных в военкомат, где его уговаривали заключить контракт.

Во-вторых, это перевод на контракт срочников. Каждый срочник может заключить контракт, который пойдет в зачет срочки.

В-третьих, это продолжающийся набор в тюрьмах, в основном, в отряды Шторм вместо Вагнера.

Все эти три метода позволяют увеличить численность личного состава в рядах вооруженных сил, и они все задействованы. Но есть еще один метод, это уже мобилизованные, которым предлагают заключить контракт. Таки люди тоже идут в статистику, но вот на общее число солдат они не влияют.

Есть и менее очевидные способы, вот новость прям с Пикабу. Когда в ходе недавних массовых рейдов среди мест компактного проживания граждан, недавно получивших паспорт, выявляли тех, кто еще не стал на учет в военкомат и помогали им это сделать прям с доставкой к месту оформления документов.

Успехи скрытой мобилизации
Вот теперь о цифрах, как ни странно, они нам известны. Бывший президент, Дмитрий Медведев, который как раз и курирует этот процесс, озвучил, что с 1 апреля по август на контракт записалась 231 тысяча человек.

Это довольно много и почти догоняет общее число мобилизованных в первую волну. То есть, по факту, перекрывает потребности во второй волне мобилизации. И да, наверняка, люди сразу скажут о вот тех самых приписках, когда уже мобилизованных переводят на контракт, учитывают в статистике, но фактическое положение дел не меняется. К сожалению, мы не знаем, сколько таких людей среди этих 231 тысячи есть.

Но у нас есть взгляд и с другой стороны. Скрытую мобилизацию в РФ совсем недавно комментировал представитель ГУР Украины, он подтвердил, что «разными методами» в месяц мобилизуется 20 тысяч человек. То есть, даже с учетом всех манипуляций, поток новобранцев в зону СВО достаточно большой, в течение года это и будут те самые примерно 300 тысяч человек, которые были в первой волне.

Промежуточные выводы
Вторая волна мобилизации в России уже идет и идет достаточно успешно. Даже при мобилизации 20 тысяч человек в месяц, это в общем позволяет руководству РФ отказаться от проведения отдельной второй волны частичной мобилизации и закрыть все потребности за счет добровольцев (условно, все мы понимаем, что часть людей заключает контракт добровольно-принудительно).

Также вспоминаем, что я написал в начале – указ о частичной мобилизации юридически еще действует. Можно предположить, что при недостаточном потоке новобранцев будет проведена не вторая волна по образу первой, а еще включен еще один механизм. Начнут рассылать повестки ежемесячно в небольшом количестве, уже прямо призывая не 300 тысяч человек за раз, а, скажем, по 10 тысяч человек в месяц.

Именно так происходит на Украине. Процесс рассылки и раздачи повесток идет постоянно без каких-либо волн. Повестка – это вызов на «уточнение данных», где уже вместо нее вручают мобилизационное предписание, по украинской практике, даже не выпуская из военкомата. И напомню, что в России рассылаются похожие повестки и сейчас, только с целью уговорить заключить контракт, но при необходимости, просто начнут сразу давать предписания.

Юридические изменения
Между тем, Госдума ввела ряд изменений в законодательство, которые тоже серьезно всколыхнули общество. Во-первых, это увеличение возраста запаса, во-вторых, увеличение возраста призыва на срочную службу, в-третьих, штрафы для уклонистов.

Из этих изменений делается вывод, что мобилизация готовится осенью. Но это не совсем так, первые два пункта точно никак не могут повлиять на осеннюю мобилизацию. Возраст учета вообще вводится поэтапно, каждый год на один год. А возраст призыва увеличивается с 2024-го, но не с 2023-го года. На мой взгляд, это не подготовка к СВО вообще, а подготовка к вероятной эскалации конфликта. Я уже как-то говорил, что мы, возможно, находимся в 1938-м или в 1913-м году, просто этого еще не знаем. А лет через 20 будем спорить, является ли Российско-Украинская война отдельным конфликтом или это один из театров действий Третьей мировой.

Остается третий пункт, ужесточение штрафов для уклонистов. Кто-то это воспринял как легитимизацию взятки за уклонение, но ведь штраф не освобождает от обязанности явиться в военкомат по повестке.

Я считаю, что тут смысл в другом. Чуть выше я писал, что один из источников пополнения личного состава в зоне СВО – это срочники. И здесь тоже есть изменения, срочник сможет заключить контракт через месяц после призыва, а раньше мог только через три.

И вот эти все изменения направлены на то, чтобы упростить осенний призыв на срочную служба, а уже на срочке начнут обрабатывать призывников на тему подписания контракта. И здесь, конечно, к кому-то могут применить кнут, кто-то поведется на пряник в виде, в первую очередь, зарплаты. Штраф этому только поможет, ведь штрафник может записаться на контракт и с нескольких зарплат с ним расплатиться.

Есть и еще одно не самое заметное изменение. С недавнего времени, иностранцы могут заключить контракт с ВС РФ на один год, а затем получить гражданство по упрощенной процедуре.

Логические «против»
Кроме того, что реальные и юридические действия руководства России нацелены на то, чтобы избежать проведения второй волны частичной мобилизации, есть еще и объективные причины, почему ее не стоит проводить, во всяком случае, сейчас.

Проведений первой волны выявило ряд проблем, которые такой метод создает. И здесь даже можно обойтись без пруфов, просто само по себе очевидно, что одномоментный призыв в армию 300 тысяч человек за 1,5 месяца сильно перегружает все тыловые службы. Так, например, призыв на срочку – это 120-150 тысяч человек в течение 3-х месяцев.

При этом, на Украине, например, при всеобщей мобилизации отменили срочную службу, но в России при частичной – нет. И в прошлом году это привело к тому, что срок призыва на срочку пришлось сдвинуть на месяц.

Зима и поздняя осень – это неудобное время для подготовки личного состава. В отличие от срочников, которые, условно, будут «красить траву», эти люди отправляются на войну и нужно постараться их хоть как-то обучить. Зимой это не слишком удобно из-за погоды, температуры и всего остального. Если на полигоне грязь, в которой танк застрянет, обучения не получится.

Конечно, возникает правильный вопрос, если все эти проблемы очевидны даже с дивана, зачем тогда проводилась первая волна? Она была продиктована военной необходимостью, когда с ней очень долго тянули, а потом вынуждены были добирать людей на войну в условиях, что они нужны были еще на вчера (на начало сентября в Харьковской области).

Сейчас же ситуация на фронте явно не диктует срочной необходимости добирать людей. В моем прошлом посте я показал, что Россия проводит успешную оборонительную операцию, нет признаков обрушения фронта или возможных скорых успехов ВСУ. При том, у России хватает сил не только обороняться, но еще и наступать на другом фланге на Купянск.

Вопрос ротации
Еще один часто поднимаемый вопрос, это то, что мобилизация необходима для ротации, а то и для демобилизации частично мобилизованных. Потому что они уже почти год как воюют, а хотят по домам. Ну или хотя бы на ротацию.

Возможно, меня поправят действующие военные, но у меня сложилось мнение, что все эти нормативы по ротации, записанные в уставах, в нынешней войне пошли к одному месту с обеих сторон. В реальности, классических ротаций войск не проводит ни Россия, ни Украина, за исключением частей, понесших сильные потери. Для отдыха военнослужащим дают отпуска, я знаю лично мобилизованных, кто уже успел побывать дома, а кто-то и два раза каким-то образом, но само подразделение в тыл не отводится.

Что же касается демобилизации – ее не планируется точно и про это говорил Путин в июне на встрече с военкорами. Ему задавали этот вопрос, от ответа он ушел, сказав, что все будет зависеть от ситуации на фронте. Потому рассчитывать на демобилизацию через год я бы точно не стал. Возможно, такие процессы начнутся к концу второго года службы мобилизованных, но точно не в ближайшее время и Путин ясно дал это понять.

К тому же, среди мобилизованных активно идет агитация подписывать контракт, который на время СВО является бессрочным. Вот, возможно, как раз к концу второго года, когда уже и будут более точные расклады по ходу боевых действий с учетом вероятного наступления ВС РФ, начнут отпускать. Тогда получится, что кто-то выбыл из-за потерь, кто-то подписал контракт и никуда не денется, а вот оставшихся хотя бы частично можно будет демобилизовать. Вероятно, по каким-то критериям, в первую очередь, тех, кто постарше, у кого много детей и т.д. И при условии, что не будет эскалации конфликта с военным положением и всеобщей мобилизацией, конечно.

Общие выводы и советы
На мой взгляд, ничего не указывает на то, что в России готовится вторая волна мобилизации осенью, как пишут некоторые медийные источники. Наоборот, принимаются меры, чтобы этой волны избежать. В реальности, мобилизация уже идет, только совсем другими методами, не теми, что были задействованы прошлой осенью.

При этом, напомню, что у меня нет инсайдов и я не принимаю решения, за чужую глупость я не в ответе. Может быть, я не прав и российское руководство решит зачем-то еще раз прыгнуть на мобилизационные грабли, но вроде я постарался обосновать, почему этого не должны сделать.

В реальности же, если в МО РФ видят, что поток контрактников недостаточен для закрытия потребностей в живой силе, я думаю, что будет задействован механизм настоящей скрытой мобилизации – рассылка повесток ежемесячно без широкой огласки и без резких «волн», чтобы не напрягать армейский тыл и максимально скрывать это от общества.

То есть, если лично вы опасаетесь быть мобилизованным, я бы дал совет не ждать повторения событий прошлого года, а внимательно следить за новостями. Конечно же, не федеральными, а местными, особенно, неформальными СМИ типа групп в Телеграме и ВК, где выкладывают локальные новости. Слушать, что говорит окружение, коллеги и даже читать оппозиционные СМИ, они точно напишут. И обращать внимание не на сам факт рассылки повесток, их постоянно рассылают для сверки данных, а вот на случаи, когда человек пошел, а ему там мобилизационное предписание выписали. Вот когда такое заметите – вот тогда собирайтесь на дачу или в Верхний Ларс, куда запланировано.

В целом же, некая масштабная мобилизация даже в виде волн – возможна, но для нее должны сложиться объективные обстоятельства непосредственно на фронте. И ясны они будут ближе к лету 2024-го года.

Мой канал в ТГ, в нем выходит намного больше разных постов, никаких репостов, исключительно мои авторские тексты: https://t.me/artjockey


#[моё] #Политика #Спецоперация #Война на Украине #Министерство обороны #Мобилизация #Война #Частичная мобилизация #Россия #Призыв #Длиннопост"""

In [ ]:
response = ce.invoke({
    CONTEXT_INPUT_TEXT: TEXT,
    CONTEXT_INPUT_POST_TIME: datetime.now(),
    CONTEXT_INPUT_GOALS: "Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions.",
    CONTEXT_INPUT_CURRENT_TIME: datetime.now(),
    CONTEXT_INPUT_USERNAME: "alex4321",
    CONTEXT_INPUT_CHARACTER: "you are a programmer, 29 y.o. male, Russian citizen",
})
print(response[CONTEXT_OUTPUT_CONTEXT])

[tags] Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост [/tags]

- Автор обсуждает возможность проведения мобилизации в России осенью: "будет ли мобилизация, если да – то когда."
- Автор упоминает о первой волне мобилизации, которая началась прошлой осенью, и с 21 сентября по 31 октября было призвано 318 тысяч человек официально. "Она началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально."
- Автор подчеркивает, что мобилизация была необходима для достижения численного паритета с ВСУ: "Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил."
- Автор утверждает, что вместо второй волны мобилизации планируется набрать 400 тысяч людей на контрактную службу: "вместо второй волны мобилизации предполагается до конца года набрать 400 тысяч людей на контрактную службу. Вот эти новые контрактник

In [ ]:
import nbdev; nbdev.nbdev_export()